In [1]:
import sys
sys.path.insert(0,'C:\\masters_thesis_code\\')

"""
Magic commands
"""

%load_ext autoreload
%autoreload 2

"""
Load relevant packages
"""

import numpy as np

"""
Own packages
"""

# from codelib.statistics.moments import corr_to_cov_matrix

'\nOwn packages\n'

In [2]:
np.mean(np.array([2,2]))

2.0